# Testing multithread efficiency - 11 Mar 2022

Function for running model speed test:

In [2]:
cd /home/wjm41/symlinks/rds/datasets/Ugis

/rds-d2/user/wjm41/hpc-work/datasets/Ugis


In [3]:
from src.utils import get_device

device = get_device()
device

'cuda'

In [4]:
import argparse
import time

from src.parsing import add_io_args, add_data_args, add_optim_args

def test_model_speed(model, epochs=1, batch_size=16384, minibatch_size=32):
    parser = argparse.ArgumentParser()
    parser = add_io_args(parser)
    parser = add_data_args(parser)
    parser = add_optim_args(parser)

    parser.add_argument('-time_forward_pass', action='store_true',
                        help='if True, will log the time taken for a forward pass a batch.')
    args = parser.parse_args(['-p', 'test/test_data/HIV.csv',
                              '-y_col', 'activity',
                              '-n_epochs', f'{epochs}',
                              '-batch_size', f'{batch_size}',
                              '-minibatch_size', f'{minibatch_size}',
                              #   '-time_forward_pass'
                              ])
    device = get_device()
    start_time = time.perf_counter()
    model(args, device)
    end_time = time.perf_counter()
    print(f'Total time taken for {epochs} epochs: {end_time-start_time:.1f}s')


Running the speed tests for 10 epochs:

TODO - need to benchmark by the number of steps, not simply the epoch run time

In [5]:
%config Application.log_level="INFO"

In [6]:
import sys

from src.train_and_validate import main as featurize_on_the_fly
from src.train_and_validate_collate import main as pre_featurize
from src.train_and_validate_pmap import main as pre_featurize_pmap

print('Testing featurize-on-the-fly model.')
test_model_speed(featurize_on_the_fly)
print('Testing pre-featurised model.')
test_model_speed(pre_featurize)
print('Testing pre-featurised pmap model.')
test_model_speed(pre_featurize_pmap)
# print('Testing pre-featurised multithread model.')
# test_model_speed(
#     pre_featurize_mutithread)


100%|█████████████████████████████████████| 3.00/3.00 [02:33<00:00, 51.0s/batch]
100%|█████████████████████████████████████| 3.00/3.00 [00:03<00:00, 1.31s/batch]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   32.7s
[Parallel(n_jobs=4)]: Done 13852 tasks      | elapsed:   40.3s
[Parallel(n_jobs=4)]: Done 41102 tasks      | elapsed:   56.4s
[Parallel(n_jobs=4)]: Done 41120 out of 41127 | elapsed:   56.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done 41127 out of 41127 | elapsed:   56.4s finished
100%|█████████████████████████████████████| 3.00/3.00 [00:31<00:00, 10.4s/batch]


In [ ]:
from src.train_and_validate_multithread import main as pre_featurize_mutithread

print('Testing pre-featurised multithread model.')
test_model_speed(pre_featurize_mutithread)

Testing pre-featurised multithread model.


  0%|                                            | 0.00/3.00 [00:00<?, ?batch/s][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 312 tasks      | elapsed:    0.3s
